In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [2]:
from tensorflow.examples.tutorials.mnist import input_data


In [3]:
mb_size = 32
x_dim = 784
z_dim = 64
h_dim = 64
R_dim=128
lr = 1e-3
d_steps = 3

mnist = input_data.read_data_sets('/home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/mnist.', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/mnist.\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/mnist.\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/mnist.\t10k-images-idx3-ubyte.gz
Extracting /home/runsheng/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/mnist.\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives suc

In [4]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


In [5]:
import numpy as np

In [6]:
# Discriminator Net

In [7]:
X = tf.placeholder(tf.float32, shape=[None, x_dim], name='X')

In [8]:
## x is a place holder for the input data  that can take any shape as long as it is 784 elements wide. 
#That's what none dimention means.

In [9]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [10]:
D_W1 = tf.Variable(xavier_init([x_dim, R_dim]), name='D_W1')
D_b1 = tf.Variable(tf.zeros(shape=[R_dim]), name='D_b1')

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
D_W2 = tf.Variable(xavier_init([R_dim, 1]), name='D_W2')
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2')

In [12]:
theta_D = [D_W1, D_W2, D_b1, D_b2]

In [13]:
#so we initialized the weights of the layers. b's as zeros and W with Xavier initializion(Gausian intialized with varience 
#inversely proportional to the size of the layer)

In [14]:
#Generator Net

In [15]:
Z = tf.placeholder(tf.float32, shape=[None, z_dim], name='Z')
disable_flag = tf.placeholder(tf.float32, shape=[None, 1], name = 'disable_flag')

In [16]:
#Z is a placeholder for the input noise for the generator.

In [17]:
G_W1 = tf.Variable(xavier_init([z_dim, h_dim]), name='G_W1')
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]), name='G_b1')

G_W2 = tf.Variable(xavier_init([h_dim, h_dim]), name='G_W2')
G_b2 = tf.Variable(tf.zeros(shape=[h_dim]), name='G_b2')

G_W3 = tf.Variable(xavier_init([h_dim, x_dim]), name='G_W3')
G_b3 = tf.Variable(tf.zeros(shape=[x_dim]), name='G_b3')

G_R = tf.Variable(xavier_init([z_dim, x_dim]), name='G_W3')

G_check = tf.Variable(tf.ones(shape=[1]), name='G_check')

In [18]:
theta_G1 = [G_W1, G_W3, G_b1,  G_b3, G_R]
    
theta_G2 = [G_W2, G_b2]


In [19]:
'''
def generator(z, disable_flag):
    if disable_flag==1:
        G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1) #G_h1=relu(G_W1*z+D_b1)
        G_log_prob=tf.matmul(G_h1,G_W3)+G_b3
        G_prob = tf.nn.sigmoid(G_log_prob)           #G_prob= sigmoid(G_h1*G_W3+G_b2)
        check=1;

    else:

        G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1) #G_h1=relu(G_W1*z+D_b1)
        G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)
        G_log_prob=tf.matmul(G_h2,G_W3)+G_b3
        G_prob = tf.nn.sigmoid(G_log_prob)           #G_prob= sigmoid(G_W2*G_W3+G_b2)
        check=0;
    return G_prob,check
'''
def generator(z, d_flag):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1) #G_h1=relu(G_W1*z+D_b1)
    G_h2 = d_flag*G_h1 + (1.0 - d_flag)* tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)
    G_log_prob=tf.matmul(G_h2,G_W3)+G_b3+tf.matmul(z,G_R)
    G_prob = tf.nn.sigmoid(G_log_prob)           #G_prob= sigmoid(G_h1*G_W3+G_b2)
    G_check = d_flag
    return G_prob, G_check

In [20]:
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1) #D_h1=relu(G_W1*x+D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)             #D_prob= sigmoid(D_h1*D_W2+D_b2)

    return D_prob, D_logit

In [21]:
'''Above, generator(z) takes 100-dimensional vector and returns 786-dimensional vector, which is MNIST image (28x28).
z here is the prior for the G(Z). In a way it learns a mapping between the prior space to Pdata.

The discriminator(x) takes MNIST image(s) and return a scalar which represents a probability of real MNIST image.'''


'Above, generator(z) takes 100-dimensional vector and returns 786-dimensional vector, which is MNIST image (28x28).\nz here is the prior for the G(Z). In a way it learns a mapping between the prior space to Pdata.\n\nThe discriminator(x) takes MNIST image(s) and return a scalar which represents a probability of real MNIST image.'

In [22]:
def log(x):
    return tf.log(x + 1e-8)
#loss functions

G_sample, g_chk = generator(Z,disable_flag)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

D_loss = -tf.reduce_mean(log(D_real) +  log(1. - D_fake)) #mean ( logD(x)+log( 1-D( G(z) ) ) )
G_loss = -tf.reduce_mean(log(D_fake)) #mean( D ( G ( Z ) ) ) ) we maximize logD(G(Z))) instead of minizing (log(1-D(G(Z)))



In [23]:
'''Above, we use negative sign for the loss functions because they need to be maximized, whereas TensorFlow’s optimizer can only do minimization.'''

'Above, we use negative sign for the loss functions because they need to be maximized, whereas TensorFlow’s optimizer can only do minimization.'

In [1]:
# Only update D(X)'s parameters, so var_list = theta_D
D_solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(D_loss, var_list=theta_D)
# Only update G(X)'s parameters, so var_list = theta_G
G_solver_1 = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss, var_list=theta_G1)
G_solver_2 = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss, var_list=theta_G2)

NameError: name 'tf' is not defined

In [25]:
def sample_Z(m, n):
    '''Uniform prior for G(Z)'''
    return np.random.uniform(-1., 1., size=[m, n])

In [26]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [27]:
for it in range(1000000):
    if (it<10000):
        X_mb, _ = mnist.train.next_batch(mb_size)    
        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, z_dim), disable_flag: np.ones(shape = [mb_size,1]).reshape(-1,1),G_W2:np.zeros(shape = [h_dim,h_dim]), G_b2:np.zeros(shape = [h_dim,])})
        _, G_loss_curr,check = sess.run([G_solver_1, G_loss,g_chk], feed_dict={Z: sample_Z(mb_size, z_dim), disable_flag: np.ones(shape = [mb_size,1]).reshape(-1,1), G_W2:np.zeros(shape = [h_dim,h_dim]), G_b2:np.zeros(shape = [h_dim,])})
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, z_dim), disable_flag: np.ones(shape = [16,1]).reshape(-1,1)})

    else:
        X_mb, _ = mnist.train.next_batch(mb_size)    
        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, z_dim), disable_flag: np.zeros(shape = [mb_size,1]).reshape(-1,1)})
        _, G_loss_curr,check = sess.run([G_solver_2, G_loss,g_chk], feed_dict={Z: sample_Z(mb_size, z_dim), disable_flag: np.zeros(shape = [mb_size,1]).reshape(-1,1)})
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, z_dim), disable_flag: np.zeros(shape = [16,1]).reshape(-1,1)})
    

    if it % 1000 == 0:
        
        op = open("deep_disable_layer.txt","a") 
        print('Iter: {} ; D_loss: {:.4}; G_loss: {:.4} ; disable_flag: {:}; check{}'
              .format(it, D_loss_curr, G_loss_curr, disable_flag, check[0]))
        op.write('Iter: {} ; D_loss: {:.4}; G_loss: {:.4} \n'
              .format(it, D_loss_curr, G_loss_curr)) 
        op.close() 
        #samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, z_dim)})

        fig = plot(samples)
        plt.savefig('out_deep_disable_layer/{}.png'
                    .format(str(it).zfill(3)), bbox_inches='tight')
        it += 1
        plt.close(fig)

Iter: 0 ; D_loss: 1.588; G_loss: 2.381 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[1.]
Iter: 1000 ; D_loss: 0.026; G_loss: 6.89 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[1.]
Iter: 2000 ; D_loss: 0.5276; G_loss: 3.982 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[1.]
Iter: 3000 ; D_loss: 0.3353; G_loss: 4.325 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[1.]
Iter: 4000 ; D_loss: 0.6057; G_loss: 1.757 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[1.]
Iter: 5000 ; D_loss: 0.6732; G_loss: 2.849 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[1.]
Iter: 6000 ; D_loss: 0.6375; G_loss: 2.524 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[1.]
Iter: 7000 ; D_loss: 0.6588; G_loss: 2.562 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[1.]
Iter: 8000 ; D

Iter: 65000 ; D_loss: 2.049e-08; G_loss: 17.63 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 66000 ; D_loss: 1.864e-05; G_loss: 18.19 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 67000 ; D_loss: 5.402e-08; G_loss: 18.01 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 68000 ; D_loss: 1.831e-06; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 69000 ; D_loss: 4.284e-08; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 70000 ; D_loss: 7.684e-06; G_loss: 17.98 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 71000 ; D_loss: 1.358e-06; G_loss: 18.27 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 72000 ; D_loss: 1.265e-05; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dt

Iter: 130000 ; D_loss: 5.96e-08; G_loss: 18.05 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 131000 ; D_loss: 2.421e-08; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 132000 ; D_loss: 1.118e-08; G_loss: 18.28 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 133000 ; D_loss: 1.788e-07; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 134000 ; D_loss: 6.817e-07; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 135000 ; D_loss: 0.001137; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 136000 ; D_loss: -0.0; G_loss: 18.36 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 137000 ; D_loss: 5.799e-06; G_loss: 18.34 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), d

Iter: 195000 ; D_loss: 2.235e-08; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 196000 ; D_loss: 0.0001215; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 197000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 198000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 199000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 200000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 201000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 202000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.

Iter: 261000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 262000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 263000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 264000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 265000 ; D_loss: 2.868e-07; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 266000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 267000 ; D_loss: 2.099e-06; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 268000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.

...

Iter: 724000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 725000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 726000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 727000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 728000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 729000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 730000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 731000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 73

Iter: 791000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 792000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 793000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 794000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 795000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 796000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 797000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 798000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 79

Iter: 857000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 858000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 859000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 860000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 861000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 862000 ; D_loss: 3.912e-08; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 863000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 864000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Ite

Iter: 923000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 924000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 925000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 926000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 927000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 928000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 929000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 930000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 93

Iter: 989000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 990000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 991000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 992000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 993000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 994000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 995000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 996000 ; D_loss: -0.0; G_loss: 18.42 ; disable_flag: Tensor("disable_flag:0", shape=(?, 1), dtype=float32); check[0.]
Iter: 99